### Paso 01 - Get Chunks

In [ ]:
# ----------------------------------------------------------------------------------------- #
# Intro de siempre
PATHFunciones = "/home/LBitn/Dropbox/git-repos/Spinal-Cord-HDMEA";
PATHBRWs = "/run/media/LBitn/Data/Medula/28-02-2022/BRWs";
push!( LOAD_PATH, PATHFunciones );
using JuliaTools
# ----------------------------------------------------------------------------------------- #

## Step 1: Cut chunks or segments from a BRW file

# ----------------------------------------------------------------------------------------- #
using JLD
FILESBRW = searchdir( PATHBRWs, "brw" );
@time for i = 2:length( FILESBRW );
    FILEBRW = FILESBRW[ i ];
    Variables, FILEPATHS = VariablesBRW( FILEBRW );
    PATHS = load( FILEPATHS )[ "PATHS" ];
    PATHMain = PATHS[ "PATHMain" ];
    PATHInfo = PATHS[ "PATHInfo" ];
    PATHSegmentos = joinpath( PATHMain, "Segmentos" );    
    Get_chunks( Variables, PATHSegmentos );
    println( basename( FILEBRW ), " is done." );
end
# ----------------------------------------------------------------------------------------- #

### Paso 02 - Voltage Convertion

In [ ]:
# ----------------------------------------------------------------------------------------- #
# Intro de siempre
PATHFunciones = "/home/LBitn/Dropbox/git-repos/Spinal-Cord-HDMEA";
PATHBRWs = "/run/media/LBitn/Data/Medula/28-02-2022/BRWs";
push!( LOAD_PATH, PATHFunciones );
using JuliaTools
# ----------------------------------------------------------------------------------------- #

## Step 2: Convertion to Voltage WITHOUT removing saturations

# ----------------------------------------------------------------------------------------- #
using JLD
Temp = readdir( dirname( PATHBRWs ), join = true );
WorkDirs = Temp[ readdir( dirname( PATHBRWs ) ) .!= basename( PATHBRWs ) ];
@time for i = 1:length( WorkDirs );
    PATHS = load( joinpath( WorkDirs[ i ], "Info", "Paths.jld" ) )[ "PATHS" ];
    PATHS[ "PATHSegmentos" ] = joinpath( WorkDirs[ i ], "Segmentos" );
    PATHS[ "PATHFunciones" ] = PATHFunciones;
    PATHS[ "Voltage" ] = joinpath( WorkDirs[ i ], "Voltage" ); 
    mkpath( PATHS[ "Voltage" ] ); cd( PATHS[ "Voltage" ] );
    Variables = load( joinpath( WorkDirs[ i ], "Info", "variablesBRW.jld" ) )[ "Variables" ];
    nChs = length( Variables[ "Layout" ] );
    channels = 1:nChs;
    FILESegmentos = searchdir( PATHS[ "PATHSegmentos" ], "jld" );
    @time for j = 1:length( FILESegmentos )
        FILESegmento = FILESegmentos[ j ];
        FILENoSat = replace( 
            replace( FILESegmento, "Segmentos" => "Voltage" ), "BIN" => "DIG" );
        BIN = load( FILESegmento )[ "data" ];
        BIN = Digital2Analogue( Variables, BIN );
        save( FILENoSat, "data", Float16.( BIN ) );
    end
    save( joinpath( WorkDirs[ i ], "Info", "Paths.jld" ), "PATHS", PATHS );
    println( basename( WorkDirs[ i ] ), " done" )
    
end
println( PATHBRWs, " done" )
# ----------------------------------------------------------------------------------------- #

### Paso 03.1 Detecting Discharges

In [ ]:
# ----------------------------------------------------------------------------------------- #
# Intro de siempre
PATHFunciones = "/home/isabel/Dropbox/git-repos/Spinal-Cord-HDMEA";
PATHBRWs = "/run/media/isabel/Data/Medula/28-02-2022/BRWs";
push!( LOAD_PATH, PATHFunciones );
using JuliaTools
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
using JLD, Plots
Aux = readdir( dirname( PATHBRWs ), join = true );
WorkDirs = Aux[ readdir( dirname( PATHBRWs ) ) .!= basename( PATHBRWs ) ];
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
p = 0.90;
@time for i = 1:length( WorkDirs )
    Variables = load( joinpath( WorkDirs[ i ], "Info", "variablesBRW.jld" ) )[ "Variables" ];
    PATHS = load( joinpath( WorkDirs[ i ], "Info", "Paths.jld" ) )[ "PATHS" ];
    PATHVoltage = joinpath( WorkDirs[ i ], "Voltage" );
    FILESVoltage = searchdir( PATHVoltage, "jld" ); 
    PATHFigures = joinpath( WorkDirs[ i ], "Figures" ); mkpath( PATHFigures );
    PATHDischarges = joinpath( PATHFigures, "Discharges" ); mkpath( PATHDischarges );
    nChs = length( Variables[ "Layout" ] ); # total number of channels (...4096)
    sparsity_matrix = zeros( length( FILESVoltage ), 3 ); # Array n segments rows x 3 columns
    # Array 4096 rows x n segments columns
    variabilidades = zeros( nChs, length( FILESVoltage ) ); 
    channels = 1:nChs;
@time for j = 1:length( FILESVoltage );
        FILEVoltage = FILESVoltage[ j ];
        data = Float64.( load( FILEVoltage )[ "data" ] ); 
        V = zeros( Int, nChs ); # preallocation
        [ V[ k ] = length( unique( data[ k, : ] ) ) for k in channels ];
        FILEfigure = joinpath( 
            PATHDischarges, replace( basename( FILEVoltage ), "jld" => "png" ) );
        FIGURE = Zplot( V, "W", :bluesreds );
        FIGURE = plot!( cbar = :none, title = basename( FILEfigure )[ 1 : ( end - 4 ) ] );
        savefig( FIGURE, FILEfigure );
        xmean, Δx, C1, C2 = MeanΔxCI( V, p ); AboveCI = findall( V .>= C1 ); 
        grupos, loose = Get_Groups( AboveCI );
        sparsity_matrix[ j, : ] = [ 
            Density( grupos ), length( AboveCI ), Sparsity( length( grupos ), nChs ) ];
        # V vector of each segment into an array
        variabilidades[ :, j ] = V;
        println( "file: ", lpad( j, 2, "0" ), " done" );
    end
    local Aux = joinpath( WorkDirs[ i ], "Info", "Reparations.jld" );
    save( Aux, "Sparsity", sparsity_matrix, "Variability", variabilidades );
    println( "file: ", lpad( i, 2, "0" ), "/", length( WorkDirs ), " done" );
end
# ----------------------------------------------------------------------------------------- #

### Paso 03.2 Removing Discharges

In [ ]:
# ----------------------------------------------------------------------------------------- # 
# Intro de siempre
PATHFunciones = "/home/isabel/Dropbox/git-repos/Spinal-Cord-HDMEA";
PATHBRWs = "/run/media/isabel/Data/Medula/28-02-2022/BRWs";
push!( LOAD_PATH, PATHFunciones );
using JuliaTools
# ----------------------------------------------------------------------------------------- #
using JLD, Statistics, StatsBase
AUX = readdir( dirname( PATHBRWs ), join = true );
WorkDirs = AUX[ readdir( dirname( PATHBRWs ) ) .!= basename( PATHBRWs ) ];
# ----------------------------------------------------------------------------------------- #
# ----------------------------------------------------------------------------------------- #
i = 1
Variables = load( joinpath( WorkDirs[ i ], "Info", "variablesBRW.jld" ) )[ "Variables" ];
PATHVoltage = joinpath( WorkDirs[ i ], "Voltage" );
FILESVoltage = searchdir( PATHVoltage, "jld" ); 
FILEReparations = joinpath( WorkDirs[ i ], "Info", "Reparations.jld" );
Reparations = load( FILEReparations );
variability = Reparations[ "Variability" ]; sparsity = Reparations[ "Sparsity" ];
SamplingRate = Variables[ "SamplingRate" ];
nChs = length( Variables[ "Layout" ] ); # total number of channels (...4096)
channels = collect( 1:nChs );
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
### CONSTANTES
antes = 25; # ms
SegmentDuration = 5; # ms
σ = 4; # as in σ*thr
# Input = sort( [ 73, 72, 45, 18 ] ); # para Phase_01-28-Feb-2022
@time Potential_Discharge = TimesPotentialDischarge( variability );
j = 1
bin = Potential_Discharge[ j ];
@time data = Float64.( load( FILESVoltage[ bin ] )[ "data" ] ); 
binsize = size( data, 2 );
@time data, earth, MinVoltSatChannels = SaturacionPositiva( Variables, data );
@time  data, DiscartedChannels = SaturacionNegativaTemporal( Variables, data );
DiscartedChannels = vcat( earth, DiscartedChannels );
# metodo 1. 
AllSegments = [ ];
AllThs = [ ];
datafilt = copy( data );
datafilt[ DiscartedChannels, : ] .= 0;
k = 1
@time while k <= nChs
    if isempty( findall( k .== DiscartedChannels ) ) 
        canal = data[ k, : ];
        MUA = FiltroMUAremez( Variables, canal );
        STh = findall( MUA .> donoho( MUA )*σ ); ITh = findall( MUA .< -donoho( MUA )*σ );
        AllTh = sort( union( ITh, STh ) );
        if !isempty( AllTh )
            temp01 = findall( diff( AllTh ) .> mean( diff( AllTh ) ) ); 
            temp01 = vcat( temp01, length( AllTh ) );
            Temp01 = AllTh[ temp01 ];
            temp02 = vcat( 1, temp01[ 1:( end - 1 ) ] .+ 1 );
            Temp02 = AllTh[ temp02 ];
            SegmentosCompletos = SegmentsComplet( 
                Temp02, antes, SegmentDuration, SamplingRate, binsize, MUA );
            if !isempty( SegmentosCompletos )
                push!( AllSegments, SegmentosCompletos )
            else
                push!( AllSegments, [ 0 ] )
            end
            push!( AllThs, AllTh )
        else
            push!( AllSegments, [ 0 ] )
            push!( AllThs, [ 0 ] )
        end
        datafilt[ k, : ] = MUA;
        k = k + 1
    else
        push!( AllSegments, [ 0 ] )
        push!( AllThs, [ 0 ] )
        k = k + 1
    end
end
ΔT = 500 # ms
A, B = ΔV( Variables, data, ΔT, DiscartedChannels );
W = log.( Int.( variability[ :, bin ] ) ); # para detectar los eventos de saturacion
Todos, t = Thresholding( W );
SatChannelsVar = sum( Todos, dims = 1 );
Temp = findall( vec( SatChannelsVar .<= 1000 ) );
NewW = sum( Todos[ :, Temp ], dims = 2);
NewT, Newt = Thresholding( Float64.( NewW ) );
SatChannelsVar = findall( vec( NewW .>= mode( Newt ) ) );
SatChannelsVarFillend = FillingHolesCrux( SatChannelsVar );
@time grupos, loose = Get_Groups( SatChannelsVarFillend );
Channels4Repair = vcat( grupos... ); Channels4RepairM1 = copy( Channels4Repair );
# Noise-adaptive Optimal Thresholding
# Ideally 4/√3, but we are looking for supra supra threshold events, so a larger σ is needed.
σ = 6; 
channels = 1:nChs;
allframes = [ ];
for j = 1:size( data, 1 )
    # Only positive side of the thresholding, to avoid detecting possible spikes
    thr = σ*donoho( data[ j, : ] ); 
    upperframes = findall( data[ j, : ] .> thr );
    push!( allframes, upperframes )   
end
FirstDetection = channels[ length.( allframes ) .>= mean( length.( allframes ) ) ];
allsatframes = vcat( allframes[ FirstDetection ]...); Temporal = countmap( allsatframes );
canal_virtual = zeros( Int, size( data, 2 ) );
k = Int.( keys( Temporal ) ); v = Int.( values( Temporal ) );
for i = 1:length( k )
    canal_virtual[ k[ i ] ] = v[ i ];
end
thr = 2*σ*donoho( canal_virtual ); Finals = findall( canal_virtual .>= thr );
upstairs = [ ]; downstairs = [ ];
for i = 1:nChs
    A = σ*donoho( data[ i, : ] ); B = -σ*donoho( data[ i, : ] );
    a = findall( data[ i, Finals ] .> A ); b = findall( data[ i, Finals ] .< B );
    if !isempty( a ) || !isempty( b )
        if !isempty( a )
            push!( upstairs, a )
        else
            push!( upstairs, 0 )
        end
        if !isempty( b )
            push!( downstairs, b )
        else
            push!( downstairs, 0 )
        end
    elseif isempty( a ) && isempty( b )
        push!( downstairs, 0 )
        push!( upstairs, 0 )
    end
end
SatChannelsThr = findall( upstairs .!= 0 );
SatChannelsThrFillend = FillingHolesCrux( SatChannelsThr );
@time grupos, loose = Get_Groups( SatChannelsThrFillend );
Channels4Repair = vcat( grupos... ); Channels4RepairM2 = copy( Channels4Repair );
NewCh4R = union( Channels4RepairM1, Channels4RepairM2 );
NewCh4R = FillingHolesCrux( NewCh4R );
### CONSTANTES
antes = 25; # ms
SegmentDuration = 5; # ms
σ = 4; # as in σ*thr
AllSegments = [ ];
AllThs = [ ];
@time for k = 1:length( channels )
    canal = data[ k, : ];
    MUA = FiltroMUAremez( Variables, canal );
    datafilt[ k, : ] = MUA;
end
@time for k = 1:length( NewCh4R )
    ch = NewCh4R[ k ];
    MUA = datafilt[ ch, : ];
    STh = findall( MUA .> donoho( MUA )*σ ); ITh = findall( MUA .< -donoho( MUA )*σ );
    AllTh = sort( union( ITh, STh ) );
    if !isempty( AllTh )
        temp01 = findall( diff( AllTh ) .> mean( diff( AllTh ) ) ); 
        temp01 = vcat( temp01, length( AllTh ) );
        Temp01 = AllTh[ temp01 ];
        temp02 = vcat( 1, temp01[ 1:( end - 1 ) ] .+ 1 );
        Temp02 = AllTh[ temp02 ];
        SegmentosCompletos = SegmentsComplet( 
            Temp02, antes, SegmentDuration, SamplingRate, binsize, MUA );
        if !isempty( SegmentosCompletos )
            push!( AllSegments, SegmentosCompletos )
        else
            push!( AllSegments, [ 0 ] )
        end
        push!( AllThs, AllTh )
    else
        push!( AllSegments, [ 0 ] )
        push!( AllThs, [ 0 ] )
    end
end
nopes = findall( Bool.( length.( AllSegments ) .% 2 ) );
deleteat!( AllSegments, nopes )
deleteat!( AllThs, nopes )
deleteat!( NewCh4R, nopes )
nframes = ms2frames( SegmentDuration, SamplingRate ) + 1;
AllSegmentFrames = zeros( Int, sum( Int.( length.( AllSegments ) ./ 2 ) ), nframes );
counter = 1
for k = 1:length( AllSegments )
    temp = AllSegments[ k ];
    for l = 1:size( temp, 1 )
        AllSegmentFrames[ counter ,: ] = 
            collect( AllSegments[ k ][ l, 1 ]:AllSegments[ k ][ l, 2 ] );
        counter = counter + 1;
    end
end
Cuanto = Int.( values( countmap( vcat( AllSegmentFrames... ) ) ) );
Donde = Int.( keys( countmap( vcat( AllSegmentFrames... ) ) ) );
CanalVirtual = zeros( Int, size( data, 2 ) ); CanalVirtual[ Donde ] = Cuanto;
NewTh = donoho( CanalVirtual )*10;
FinalFrames = findall( CanalVirtual .>= NewTh );
NsegmentsXchannel = Int.( length.( AllSegments ) ./ 2 );
for k = 2:length( NsegmentsXchannel )
    NsegmentsXchannel[ k ] = NsegmentsXchannel[ k - 1 ] + NsegmentsXchannel[ k ];
end
FinalSegments = [ ];
for k = 1:size( AllSegmentFrames, 1 ) 
    if !isempty( findall( AllSegmentFrames[ k, : ] .∈ [ FinalFrames ] ) )
        push!( FinalSegments, k )
    end
end
NsegmentsXchannel = vcat( 0, NsegmentsXchannel );
FinalChannels = [ ]
for k = 1:length( FinalSegments )
    for nseg = 1:( length( NsegmentsXchannel ) - 1 )
        if NsegmentsXchannel[ nseg ] <= FinalSegments[ k ] <=  NsegmentsXchannel[ nseg + 1 ]
            push!( FinalChannels, nseg )
        end
    end
end
FinalChannels = NewCh4R[ FinalChannels ];
DataReparado = copy( data );
FinalChannelsUnique = unique( FinalChannels );
nframesAreparar = length( FinalFrames );
aux = setdiff( collect( 1:length( canalAreparar ) ), FinalFrames );
@time for k = 1:length( FinalChannelsUnique )
    canal = FinalChannelsUnique[ k ];
    canalAreparar = DataReparado[ canal, : ];
    aux = setdiff( collect( 1:length( canalAreparar ) ), FinalFrames );
    canalAreparar[ FinalFrames ] = canalAreparar[ sample( aux, nframesAreparar ) ];
    DataReparado[ canal, : ] = canalAreparar;
end
# ----------------------------------------------------------------------------------------- #